<a href="https://colab.research.google.com/github/zouzanope/income_forecast/blob/main/income_forecast.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Подключение к БД

In [ ]:
!mkdir -p ~/.postgresql && \
wget "https://storage.yandexcloud.net/cloud-certs/CA.pem" \
    --output-document ~/.postgresql/root.crt && \
chmod 0600 ~/.postgresql/root.crt
!sudo apt update && sudo apt install --yes python3 python3-pip && \
!pip3 install psycopg2-binary

--2023-02-15 17:25:40--  https://storage.yandexcloud.net/cloud-certs/CA.pem
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3579 (3.5K) [application/x-x509-ca-cert]
Saving to: ‘/root/.postgresql/root.crt’

/root/.postgresql/r 100%[===================>]   3.50K  --.-KB/s    in 0s      

2023-02-15 17:25:40 (499 MB/s) - ‘/root/.postgresql/root.crt’ saved [3579/3579]

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu2004/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu focal-sec

Подключение библиотек и выгрузка датасета

In [4]:
import psycopg2
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings("ignore")

conn = psycopg2.connect("""
    host=rc1b-70q90cunwkzx49mj.mdb.yandexcloud.net
    port=6432
    sslmode=verify-full
    dbname=db_clients
    user=mlhs_student
    password=mlhs_student
    target_session_attrs=read-write
""")

query = "SELECT * from public.adult_income"
data = pd.read_sql_query(query, conn)

conn.close()

data.head(9)

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K
5,34,Private,198693,10th,6,Never-married,Other-service,Not-in-family,White,Male,0.0,0.0,30.0,United-States,<=50K
6,29,?,227026,HS-grad,9,Never-married,?,Unmarried,Black,Male,0.0,0.0,40.0,United-States,<=50K
7,63,Self-emp-not-inc,104626,Prof-school,15,Married-civ-spouse,Prof-specialty,Husband,White,Male,3103.0,0.0,32.0,United-States,>50K
8,24,Private,369667,Some-college,10,Never-married,Other-service,Unmarried,White,Female,0.0,0.0,40.0,United-States,<=50K


In [5]:
data.shape

(9615, 15)

In [7]:
data.dtypes

age                  int64
workclass           object
fnlwgt               int64
education           object
educational-num      int64
marital-status      object
occupation          object
relationship        object
race                object
gender              object
capital-gain       float64
capital-loss       float64
hours-per-week     float64
native-country      object
income              object
dtype: object

## Сохранение данных

In [8]:
data.to_csv("clients_class.csv", index=False)

### Разведочный анализ данных

## 1. Оценка качества данных
* пропущенные значения
* ошибочные значения

In [9]:
len(data)

9615

##### Посмотрим, есть ли в данных пропущенные значения.

In [10]:
data.isna().sum()

age                0
workclass          0
fnlwgt             0
education          0
educational-num    0
marital-status     0
occupation         0
relationship       0
race               1
gender             1
capital-gain       1
capital-loss       1
hours-per-week     1
native-country     1
income             1
dtype: int64

Заполним пропуски в столбцах: race, gender, capital-gain, capital-loss, hours-per-week, native-country, income

In [35]:
# data['gender']
# data['income'].value_counts()
# median_cap_g = data['capital-gain'].median
# median_cap_l = data['capital-loss'].median
# median_hou_w = data['hours-per-week'].median

data["race"].fillna('White', inplace=True)
data["gender"].fillna('Male', inplace=True)
data["capital-gain"].fillna(data['capital-gain'].median, inplace=True)
data["capital-loss"].fillna(data['capital-loss'].median, inplace=True)
data["hours-per-week"].fillna(data['hours-per-week'].median, inplace=True)
data["native-country"].fillna('United-States', inplace=True)
data["income"].fillna('<=50K', inplace=True)

# data.isna().sum()
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0.0,0.0,40.0,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0.0,0.0,50.0,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0.0,0.0,40.0,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688.0,0.0,40.0,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0.0,0.0,30.0,United-States,<=50K


### Основные числовые характеристики

In [37]:
# data['income'].value_counts()
data.describe()

,age,fnlwgt,educational-num
count,9615.000000,9.615000e+03,9615.000000
mean,38.732397,1.882881e+05,10.080291
std,13.842364,1.051318e+05,2.562674
min,17.000000,1.376900e+04,1.000000
25%,28.000000,1.154725e+05,9.000000
50%,37.000000,1.778390e+05,10.000000
75%,48.000000,2.374070e+05,12.000000
max,90.000000,1.490400e+06,16.000000


In [38]:
data.describe(include='object') # включение категориальных колонок

,workclass,education,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
count,9615,9615,9615,9615,9615,9615,9615,9615.0,9615.0,9615.0,9615,9615
unique,9,16,7,15,6,5,2,105.0,77.0,88.0,41,2
top,Private,HS-grad,Married-civ-spouse,Prof-specialty,Husband,White,Male,0.0,0.0,40.0,United-States,<=50K
freq,6574,3121,4307,1222,3800,8256,6410,8826.0,9165.0,4494.0,8682,7357


In [39]:
corr = data.corr() # построение матрицы корреляций

corr.style.background_gradient(cmap='coolwarm')

,age,fnlwgt,educational-num
age,1.000000,-0.081981,0.006963
fnlwgt,-0.081981,1.000000,-0.024444
educational-num,0.006963,-0.024444,1.000000


### Получается нехватка данных, слишком много атрибутов содержат категориальные признаки
